In [24]:
#show all output in each cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
#import packages
import pandas as pd
import numpy as np
#read in dataset
df = pd.read_csv('hbcu.csv')
#print out column names
print(df.columns)

Index(['Name', 'Address', 'Website', 'Type', 'Awards offered',
       'Campus setting', 'Campus housing', 'Student population',
       'Undergraduate students', 'Graduation Rate', 'Transfer-Out Rate',
       'Cohort Year *', 'Net Price **', 'Largest Program', 'IPEDS ID',
       'OPE ID'],
      dtype='object')


In [ ]:
#get data type for each column
df.info()

In [ ]:
#work on dataset
#create new highest degree column
df['Awards offered'].unique()

In [37]:
#create column for highest degree offered
print(df.columns)

Index(['Name', 'Address', 'Website', 'Type', 'Awards offered',
       'Campus setting', 'Campus housing', 'Student population',
       'Undergraduate students', 'Graduation Rate', 'Transfer-Out Rate',
       'Cohort Year *', 'Net Price **', 'Largest Program', 'IPEDS ID',
       'OPE ID'],
      dtype='object')


In [49]:
#fill in NaN values
df['Awards offered'].isna().sum()
df['Awards offered'].fillna('Missing', inplace=True)
df['Awards offered'].isna().sum()

3

0

In [64]:
df['Awards offered'].str.contains('Doctor')



0       True
1       True
2      False
3       True
4       True
       ...  
100     True
101     True
102    False
103    False
104    False
Name: Awards offered, Length: 105, dtype: bool